In [1]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json

import glob
import tqdm

pd.set_option("max.columns", 131)

#https://strftime.org/
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv("labels_curso - to_label_2.csv", index_col=0).dropna(subset=["y","watch-title"])

In [3]:
df.duplicated().mean()

0.0

In [4]:
df.duplicated(['watch-title']).mean()

0.003108003108003108

In [5]:
df.shape

(1287, 16)

In [6]:
df = df[df['watch-time-text'].notnull()]
df = df[df['watch-time-text'].str.contains('horas')==False]
df.shape

(1274, 16)

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [8]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['watch-title']

## 1. Data Cleaner

In [9]:
clean_date = df['watch-time-text'].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)")
clean_date[0] = clean_date[0].map(lambda x: "0"+x[0] if len(x) == 1 else x)
#clean_date[1] = clean_date[1].map(lambda x: x[0].upper()+x[1:])

mapa_meses = {"jan": "Jan",
              "fev": "Feb",
              "mar": "Mar", 
              "abr": "Apr", 
              "mai": "May", 
              "jun": "Jun",
              "jul": "Jul",
              "ago": "Aug", 
              "set": "Sep", 
              "out": "Oct", 
              "nov": "Nov",
              "dez": "Dec"}

clean_date[1] = clean_date[1].map(mapa_meses)

clean_date = clean_date.apply(lambda x: " ".join(x), axis=1)
clean_date.head()
df_limpo['date'] = pd.to_datetime(clean_date, format="%d %b %Y")

## 2. Views Cleaner

In [10]:
views = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".", "").fillna(0).astype(int)
df_limpo['views'] = views

## 3. Features

In [11]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()


In [12]:
features['publish_time'] = (pd.to_datetime('2020-06-01') - df_limpo['date']) / np.timedelta64(1,'D')
features['publish_time'] = features['publish_time'].replace(0,1)
features['views'] = df_limpo['views']
features['day_views'] = features['views'] / features['publish_time']
features = features.drop(['publish_time'], axis=1)

In [13]:
mask_train = df_limpo['date'] < "2019-09-01"
mask_val = df_limpo['date'] >= "2019-09-01"

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((636, 2), (638, 2), (636,), (638,))

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2)
#title_vec = TfidfVectorizer(min_df=2,ngram_range=(1,2))
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)


In [16]:
title_bow_train.shape

(636, 474)

In [17]:
from scipy.sparse import hstack, vstack

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [26]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((636, 476), (638, 476))

## 4. Random Forest

In [61]:
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, min_samples_leaf = 2, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=6, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [62]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

In [63]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [64]:
average_precision_score(yval, p)

0.18187105538013426

In [65]:
roc_auc_score(yval, p)

0.638463505412393

ap 0.14515110965950923 auc 0.6286993940458417 - min_df=2, min_sample_leaf=1 

ap 0.18187105538013426auc 0.638463505412393 min_sample_leaf=2

ap 0.14627188339357072 auc 0.6273235561019876 n_estimators=100, min_sample_lead=2

ap 0.14662005939653658 auc 0.6072129036035245 min_df=1,ngram_range=(1,2)

ap 0.1538379848663552 auc 0.6338807411961007 min_df=2,ngram_range=(1,2)

RF: ap 0.18187105538013426auc 0.638463505412393 min_sample_leaf=2

## 5.Light GBM

In [67]:
from lightgbm import LGBMClassifier

In [68]:
mdl = LGBMClassifier(random_state=0, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
               colsample_bytree=1.0, importance_type='split', learning_rate=0.1,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=100, n_jobs=6, num_leaves=31,
               objective=None, random_state=0, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [69]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

/home/rodrigorezendes/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [70]:
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.1400434244002939, 0.624683921925164)

# 6 Bayesian Optimization

In [73]:
from skopt import forest_minimize

In [74]:
def tune_lgbm(params):
    print(params)
    lr = params[0]
    max_depth = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    n_estimators = params[5]
    
    min_df = params[6]
    ngram_range = (1, params[7])
    
    title_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
    title_bow_train = title_vec.fit_transform(title_train)
    title_bow_val = title_vec.transform(title_val)
    
    Xtrain_wtitle = hstack([Xtrain, title_bow_train])
    Xval_wtitle = hstack([Xval, title_bow_val])
    
    mdl = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                         min_child_samples=min_child_samples, subsample=subsample,
                         colsample_bytree=colsample_bytree, bagging_freq=1,n_estimators=n_estimators, random_state=0, 
                         class_weight="balanced", n_jobs=6)
    mdl.fit(Xtrain_wtitle, ytrain)
    
    p = mdl.predict_proba(Xval_wtitle)[:, 1]
    
    print(roc_auc_score(yval, p))
    
    return -average_precision_score(yval, p)


space = [(1e-3, 1e-1, 'log-uniform'), # lr
          (1, 10), # max_depth
          (1, 20), # min_child_samples
          (0.05, 1.), # subsample
          (0.05, 1.), # colsample_bytree
          (100,1000), # n_estimators
          (1,5), # min_df
          (1,5)] # ngram_range

res = forest_minimize(tune_lgbm, space, random_state=160745, n_random_starts=20, n_calls=50, verbose=1)

Iteration No: 1 started. Evaluating function at random point.
[0.009944912110647982, 5, 1, 0.4677107511929402, 0.49263223036174764, 272, 3, 1]


/home/rodrigorezendes/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.6151660652896553
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.5293
Function value obtained: -0.1377
Current minimum: -0.1377
Iteration No: 2 started. Evaluating function at random point.
[0.053887464791860025, 1, 15, 0.7437489153990157, 0.8675167974293533, 549, 3, 4]


/home/rodrigorezendes/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.6034179049350797
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.5538
Function value obtained: -0.1545
Current minimum: -0.1545
Iteration No: 3 started. Evaluating function at random point.
[0.004151454520895999, 6, 20, 0.8682075103820793, 0.9491436163200662, 411, 4, 3]


...

0.5835724635622354
Iteration No: 48 ended. Search finished for the next optimal point.
Time taken: 1.1683
Function value obtained: -0.1761
Current minimum: -0.2142
Iteration No: 49 started. Searching for the next optimal point.
[0.019087344512738236, 1, 3, 0.6568054299161659, 0.46722745650919895, 200, 5, 5]


/home/rodrigorezendes/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.6205216353666506
Iteration No: 49 ended. Search finished for the next optimal point.
Time taken: 0.7609
Function value obtained: -0.1847
Current minimum: -0.2142
Iteration No: 50 started. Searching for the next optimal point.
[0.016238983884766554, 1, 1, 0.19764622784463604, 0.0686305109803104, 981, 5, 5]


/home/rodrigorezendes/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.5663834985936302
Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 1.1809
Function value obtained: -0.1222
Current minimum: -0.2142


LGBM: ap 0.2142 auc 0.6133761400119329


In [76]:
res.x

[0.027035241802913952,
 1,
 3,
 0.33397697274506905,
 0.3140136337076569,
 802,
 5,
 5]

# 7 Logistic Reg

In [77]:
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from scipy.sparse import csr_matrix

In [99]:
Xtrain_wtitle2 = csr_matrix(Xtrain_wtitle.copy())
Xval_wtitle2 = csr_matrix(Xval_wtitle.copy())

scaler = StandardScaler()
#scaler = MaxAbsScaler()


Xtrain_wtitle2[:, :2] = scaler.fit_transform(Xtrain_wtitle2[:, :2].todense())
Xval_wtitle2[:, :2] = scaler.transform(Xval_wtitle2[:, :2].todense())

#Xtrain_wtitle2 = scaler.fit_transform(Xtrain_wtitle2)
#Xval_wtitle2 = scaler.transform(Xval_wtitle2)

/home/rodrigorezendes/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/home/rodrigorezendes/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [100]:
Xval_wtitle2.shape

(638, 476)

In [122]:

mdl = LogisticRegression(C=0.5,n_jobs=6, random_state=0)
mdl.fit(Xtrain_wtitle2, ytrain)

LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=6, penalty='l2', random_state=0,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [123]:
p = mdl.predict_proba(Xval_wtitle2)[:, 1]

In [124]:
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.20804541108738267, 0.6431798164616301)

ap 0.20804541108738267 auc 0.6431798164616301 - standard
ap 0.18643772721623786 auc 0.6524419694860357 - abs